In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
#Checking missing Values
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
#Drop NaN Values
df.dropna(inplace=True)

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#Get the Independent Features
X=df.drop('label',axis=1)
#get the dependent Features
y=df['label']


In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf
#pylint: disable=import-error
from tensorflow.keras.layers import Embedding, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [12]:
#Vocabulary Size
voc_size=5000


In [13]:
#One Hot representation
messages=X.copy()

In [14]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
messages.reset_index(inplace=True)

In [17]:
##Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review= ' '.join(review)
  corpus.append(review)

In [18]:
## One Hot Representtion
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4687, 296, 468, 2998, 4662, 1918, 2195, 156, 565, 2525],
 [2374, 1513, 33, 2402, 4119, 4727, 253],
 [2007, 3102, 3340, 2353],
 [4644, 85, 3436, 1728, 1018, 1913],
 [3564, 4119, 1388, 227, 2264, 1716, 4119, 724, 544, 4714],
 [4152,
  1540,
  2506,
  4383,
  582,
  729,
  4072,
  1587,
  3721,
  4030,
  784,
  1846,
  4988,
  1764,
  253],
 [232, 3239, 4126, 1470, 2780, 4248, 1937, 1932, 1848, 1653, 1478],
 [1509, 2023, 2397, 3523, 1116, 3977, 729, 1757, 1848, 1653, 1478],
 [2127, 2233, 3283, 2515, 3262, 910, 4919, 2987, 729, 638],
 [3225, 1265, 4411, 4180, 138, 840, 3388, 3092],
 [3584, 2803, 3605, 83, 1685, 1544, 4952, 3986, 3572, 1236, 3310],
 [1728, 3305, 4662, 910, 729, 1116],
 [2577, 717, 3981, 3475, 175, 879, 712, 970, 1993],
 [4728, 2694, 811, 283, 4026, 4588, 3372, 1848, 1653, 1478],
 [2632, 1003, 4111, 658, 2726, 1848, 1653, 1478],
 [226, 4642, 3177, 4035, 4025, 1010, 4563, 2575, 477, 2877],
 [4513, 1059, 1513],
 [1679, 3017, 4499, 436, 729, 417, 4789, 253],
 [3825, 4932, 33,

In [19]:
#Embedding representation // word2vec representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  156  565 2525]
 [   0    0    0 ... 4119 4727  253]
 [   0    0    0 ... 3102 3340 2353]
 ...
 [   0    0    0 ... 1848 1653 1478]
 [   0    0    0 ... 2326 3986  122]
 [   0    0    0 ... 3426 4207 1585]]


In [20]:
# Creating the Embedding Model
embedding_vector_features = 40
# Define the model using Input layer
model = Sequential([
    Input(shape=(sent_length,)),  # Specify input shape (sentence length)
    Embedding(input_dim=voc_size, output_dim=embedding_vector_features),
    Bidirectional(LSTM(100)),
    Dense(1, activation='sigmoid')
])

In [21]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
#Train test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [26]:
#Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=32)


Epoch 1/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.8048 - loss: 0.3835 - val_accuracy: 0.9114 - val_loss: 0.2169
Epoch 2/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9461 - loss: 0.1390 - val_accuracy: 0.9147 - val_loss: 0.1972
Epoch 3/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9639 - loss: 0.0911 - val_accuracy: 0.9148 - val_loss: 0.2170
Epoch 4/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9780 - loss: 0.0620 - val_accuracy: 0.9110 - val_loss: 0.2608
Epoch 5/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9874 - loss: 0.0411 - val_accuracy: 0.9110 - val_loss: 0.2849
Epoch 6/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9918 - loss: 0.0235 - val_accuracy: 0.9054 - val_loss: 0.3664
Epoch 7/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9971 - loss: 0.0112 - val_accuracy: 0.9064 - val_loss: 0.4764
Epoch 8/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9977 - loss: 0.0095 - val_accuracy

In [27]:
#Performance metrics
y_pred=model.predict(X_test)


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [28]:
y_pred=np.where(y_pred>0.5,1,0) ##AUC, ROC Curve


In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3129,  290],
       [ 274, 2342]])

In [30]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Display the accuracy as a percentage
accuracy_percentage = accuracy * 100
print(f'Accuracy: {accuracy_percentage:.2f}%')

Accuracy: 90.65%


In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035

